# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [7]:
#!pip install pandas

In [10]:
import pandas as pd
df = pd.read_csv('data.csv',delimiter=';')

In [11]:
df.head()

,Product,Review
0,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",I thought it would be as big as small paper bu...
1,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",This kindle is light and easy to use especiall...
2,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Didnt know how much i'd use a kindle so went f...
3,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",I am 100 happy with my purchase. I caught it o...


## LLMChain

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [13]:
llm = ChatOpenAI(temperature=0.9)


In [14]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [15]:

chain = LLMChain(llm=llm, prompt=prompt)

In [16]:
product = "Amazon Kindle"
chain.run(product)

'KindleGenius'

## SimpleSequentialChain

In [17]:
from langchain.chains import SimpleSequentialChain

In [18]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [19]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [20]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [21]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"KindleWorks" or "eReaderTech" would be suitable names to describe a company that makes Amazon Kindle.
"KindleWorks: A leading e-reader manufacturer designing and producing high-quality Amazon Kindle devices for an exceptional reading experience."

> Finished chain.


'"KindleWorks: A leading e-reader manufacturer designing and producing high-quality Amazon Kindle devices for an exceptional reading experience."'

## SequentialChain

In [22]:
from langchain.chains import SequentialChain

In [23]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [24]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [25]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [26]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [27]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [30]:
review = df.Review[3]
overall_chain(review)



> Entering new SequentialChain chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-RTaSSDWLRLKfbx1k0mNjvZeD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-RTaSSDWLRLKfbx1k0mNjvZeD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht


> Finished chain.


{'Review': "I am 100 happy with my purchase. I caught it on sale at a really good price. I am normally a real book person, but I have a 1 year old who loves ripping up pages. The Kindle prevents that, it's extremely portable (it fits better in my purse than a giant book), and I have it loaded with lots of books. I finish one and start another, without having to go store. It serves all my needs. I picked this one over the Paperwhite because the price was unbeatable and the only difference that I could see was this one wasn't backlit. A simple book light from the Dollar tree solves that issue. This is my second Kindle (the first being the old Keyboard model, which I put down because I fell out of love with the keyboard. Lol) and it most likely won't be my last.",
 'English_Review': "I am 100% happy with my purchase. I managed to buy it at a really good price during a sale. I am usually a fan of physical books, but I have a one-year-old who loves tearing pages apart. The Kindle prevents t

### Exercise 1